# ECG Hybrid 1D-CNN Client Side
This code is the server part of ECG Hybrid 1D-CNN model for **multi** client and a server.

In [1]:
users = 2 # number of clients

## Import required packages

In [2]:
import os
import struct
import socket
import pickle
import time

import h5py
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import copy

In [3]:
root_path = '../../models/'

## SET CUDA

In [4]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
torch.manual_seed(777)
if device =="cuda:0":
    torch.cuda.manual_seed_all(777)

In [5]:
client_order = int(input("client_order(start from 0): "))

client_order(start from 0): 0


In [6]:
num_traindata = 13244 // users

## Define ECG dataset class

In [7]:
class ECG(Dataset):
    def __init__(self, train=True):
        if train:
            # total: 13244
            with h5py.File(os.path.join(root_path, 'ecg_data', 'train_ecg.hdf5'), 'r') as hdf:
                self.x = hdf['x_train'][num_traindata * client_order : num_traindata * (client_order + 1)]
                self.y = hdf['y_train'][num_traindata * client_order : num_traindata * (client_order + 1)]

        else:
            with h5py.File(os.path.join(root_path, 'ecg_data', 'test_ecg.hdf5'), 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

### Set batch size

In [8]:
batch_size = 32

## Make train and test dataset batch generator

In [9]:
train_dataset = ECG(train=True)
test_dataset = ECG(train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

### Size check

In [10]:
x_train, y_train = next(iter(train_loader))
print(x_train.size())
print(y_train.size())

torch.Size([32, 1, 130])
torch.Size([32])


### Total number of batches

In [11]:
total_batch = len(train_loader)
print(total_batch)

207


## Define ECG client model
Client side has only **2 convolutional layers**.

In [12]:
class Ecgclient(nn.Module):
    def __init__(self):
        super(Ecgclient, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        return x   

In [13]:
ecg_client = Ecgclient().to(device)
print(ecg_client)

Ecgclient(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (relu1): LeakyReLU(negative_slope=0.01)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu2): LeakyReLU(negative_slope=0.01)
)


In [14]:
# from torchsummary import summary

# print('ECG 1D CNN clients')
# summary(ecg_client, (1, 130))

### Set other hyperparameters in the model
Hyperparameters here should be same with the server side.

In [15]:
epoch = 20  # default
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = Adam(ecg_client.parameters(), lr=lr)

## Socket initialization

### Required socket functions

In [16]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

### Set host address and port number

In [17]:
host = input("IP address: ")
port = 10080

IP address: 192.168.83.1


## SET TIMER

In [18]:
start_time = time.time()    # store start time
print("timmer start!")

timmer start!


### Open the client socket

In [19]:
s = socket.socket()
s.connect((host, port))

In [20]:
msg = recv_msg(s)   # get epoch
rounds = msg['rounds']
local_epoch = msg['local_epoch']
client_id = msg['client_id']
msg = {
        'total_batch': total_batch,
        'train_dataset_size': len(train_dataset)
    }
send_msg(s, msg)   # send total_batch of train dataset

## Real training process

In [21]:
for r in range(rounds):
    client_weights = recv_msg(s)
    ecg_client.load_state_dict(client_weights)
    ecg_client.eval()
    for l in range(local_epoch):
        for i, data in enumerate(tqdm(train_loader, ncols=100, desc='Round '+str(r+1)+ '_' +str(l))):
            x, label = data
            x = x.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            output = ecg_client(x)
            client_output = output.clone().detach().requires_grad_(True)
            msg = {
                'client_output': client_output,
                'label': label
            }
            send_msg(s, msg)
            client_grad = recv_msg(s)
            output.backward(client_grad)
            optimizer.step()
    msg = copy.deepcopy(ecg_client.state_dict())
    send_msg(s, msg)

Round 1_0:   0%|                                                            | 0/207 [00:00<?, ?it/s]C:\Users\rlaal\anaconda3\envs\py36\lib\site-packages\torch\storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
Round 158_0: 100%|████████████████████████████████████████████████| 207/207 [00:05<00:00, 40.21it/s]


Round 320_0: 100%|████████████████████████████████████████████████| 207/207 [00:05<00:00, 39.30it/s]


Round 400_0: 100%|████████████████████████████████████████████████| 207/207 [00:05<00:00, 39.32it/s]


In [22]:
end_time = time.time()  #store end time
print("WorkingTime of ",device ,": {} sec".format(end_time - start_time))

WorkingTime of  cpu : 3305.5667338371277 sec
